In [1]:
pip install ipywidgets



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Acer\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install mediapipe


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Acer\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Acer\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import cv2
import os
import mediapipe as mp
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd
from google.protobuf.json_format import MessageToDict
import csv

In [2]:
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [3]:
folder_path = r'D:\Chord_Data'
create_folder(folder_path)

In [4]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

In [4]:
chord_shape = widgets.Dropdown(
    options=['C', 'D', 'E', 'G', 'A', 'Dm', 'Em', 'Am', 'E_Barre', 'A_Barre', 'Em_Barre', 'Am_Barre'],
    value='C',
    description='Chord:',
    disabled=False,
)

finger = widgets.Dropdown(
    options=['Index', 'Middle', 'Ring','Pinky','Angle_Open','Angle_Barre'],
    value='Index',
    description='Finger:',
    disabled=False,
)

wrong_right = widgets.Dropdown(
    options=['Right','Wrong'],
    value='Right', 
    description='Wrong/Right:',
    disabled=False,
)

display(chord_shape)
display(finger)
display(wrong_right)

Dropdown(description='Finger:', options=('Index', 'Middle', 'Ring', 'Pinky', 'Angle_Open', 'Angle_Barre'), val…

Dropdown(description='Wrong/Right:', options=('Right', 'Wrong'), value='Right')

In [47]:
video_path = 'D:\Chord_Data\Chords\\Am_Barre_test.mp4'
cap = cv2.VideoCapture(video_path)

In [34]:
video_path

'D:\\Chord_Data\\Angle_Barre\\Test_right_2.mp4'

In [7]:
def getbigclass(chord):
    classnum = 0
    chord_list=['C', 'D', 'E', 'G', 'A', 'Dm', 'Em', 'Am', 'E_Barre', 'A_Barre', 'Em_Barre', 'Am_Barre']
    for i in range(len(chord_list)):
        if chord == chord_list[i]:
            classnum = i
            break
    return classnum

In [8]:
def getinsideclass(type):
    classnum = 0
    values=['Right', 'Wrong']
    for i in range(len(values)):
        if type == values[i]:
            classnum = i
            break
    return classnum

In [9]:
x_data = []
y_data = []
landmarks_data = []

In [49]:
print(len(x_data))
print(len(y_data))
print(len(landmarks_data))

0
0
6480


In [35]:
landmarks_data = landmarks_data[:3600]
x_data = x_data[:4200]
y_data = y_data[:4200]

In [31]:
def capture_hand_images_chord(chord, num_images=540, image_size=(400, 400)):
    classnumber = getbigclass(chord)
    folder_path_chord = os.path.join(folder_path, 'Chords','Test_Img', f'{classnumber}')
    create_folder(folder_path_chord)
    count = 0
    while count < num_images:
        ret, frame = cap.read()
        framewithoutland = frame.copy()
        if not ret:
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        img_y, img_x, _ = frame.shape

        keypoint_connections = [(0,1), (1,2), (2,3), (3,4), (0,5), (5,6), (6,7), (7,8), (5,9), (9,10), (10,11), (11,12), (9,13), (13,14), (14,15), (15,16), (13,17), (17,18), (18,19), (19,20), (0,17)]
        point_draw = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = MessageToDict(hand_landmarks)["landmark"]
                decoded_landmarks = []
                for l in landmarks:
                    x, y, z = int(l['x'] * img_x), int(l['y'] * img_y), l['z']
                    decoded_landmarks.append({'x': x, 'y': y, 'z': z})

                landmarks_row = [classnumber] + [l['x']/img_x for l in decoded_landmarks] + [l['y']/img_y for l in decoded_landmarks] + [l['z'] for l in decoded_landmarks] # for csv data
                landmarks_data.append(landmarks_row)
                for i in range(21):
                    cv2.circle(frame, (decoded_landmarks[i]['x'], decoded_landmarks[i]['y']), 5, (255, 255 ,255), 5)
                for c in keypoint_connections:
                    cv2.line(frame, (decoded_landmarks[c[0]]['x'], decoded_landmarks[c[0]]['y']) , (decoded_landmarks[c[1]]['x'], decoded_landmarks[c[1]]['y']), (255, 255, 255), 5)
        
        cv2.imshow('Hand Landmarks', frame)
        cv2.waitKey(1)

        skeleton = cv2.bitwise_and(frame, cv2.bitwise_not(framewithoutland))
        skeleton = cv2.addWeighted(skeleton, 1, skeleton, 0, 50)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                min_x = min(decoded_landmarks[i]['x'] for i in point_draw)
                min_y = min(decoded_landmarks[i]['y'] for i in point_draw)
                max_x = max(decoded_landmarks[i]['x'] for i in point_draw)
                max_y = max(decoded_landmarks[i]['y'] for i in point_draw)
                real_x_min, real_y_min = int(min_x), int(min_y)
                real_x_max, real_y_max = int(max_x), int(max_y)
                if real_x_min >= 0 and real_x_max <= img_x and real_y_min >= 0 and real_y_max <= img_y:
                    hand_cropped = framewithoutland[int(real_y_min)-25:int(real_y_max)+25, int(real_x_min)-25:int(real_x_max)+25]
                    if not hand_cropped.size == 0:
                        hand_image = cv2.resize(hand_cropped, image_size)
                        image_path = os.path.join(folder_path_chord, f'hand_{count}.jpg')
                        cv2.imwrite(image_path, hand_image) # for image data
                        count += 1


In [53]:
def capture_hand_finger_wrong(finger, wrong_right, num_images=76, image_size=(400, 400)):
    classnumber = getinsideclass(wrong_right)
    folder_path_wrong_right = os.path.join(folder_path, finger, 'test_img', f'{classnumber}')
    create_folder(folder_path_wrong_right)
    count = 0
    while count < num_images:
        ret, frame = cap.read()
        framewithoutland = frame.copy()
        if not ret: 
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        img_y, img_x, _ = frame.shape

        if finger == 'Index':
            keypoint_connections = [(5, 6), (6, 7), (7, 8)]
            point_draw = [5, 6, 7, 8]
            point_draw = [6, 7, 8] # For image data
        elif finger == 'Middle':
            keypoint_connections = [(9, 10), (10, 11), (11, 12)]
            point_draw = [9, 10, 11, 12]
            point_draw = [10, 11, 12] # For image data
        elif finger == 'Ring':
            keypoint_connections = [(13, 14), (14, 15), (15, 16)]
            point_draw = [13, 14, 15, 16]
            point_draw = [14, 15, 16] # For image data
        elif finger == 'Pinky':
            keypoint_connections = [(17, 18), (18, 19), (19, 20)]
            point_draw = [17, 18, 19, 20]
            point_draw = [18, 19, 20] # For image data
        elif finger == 'Angle_Open':
            keypoint_connections = [(0, 1), (0, 5), (0, 17), (5, 17)]
            keypoint_connections = [(0, 1), (1, 5), (5, 9), (9, 13), (13, 17)] # For image data
            point_draw = [0, 1, 5, 17]
            point_draw = [0, 1, 5, 9, 13, 17] # For image data
        else:
            keypoint_connections = [(0, 1), (0, 5), (5, 6), (6, 7), (7, 8), (0, 17), (5, 17), (17, 18)]
            point_draw = [0, 1, 5, 6, 7, 8, 17, 18]
            keypoint_connections = [(5, 6), (6, 7), (7, 8)] # For image data
            point_draw = [5, 6, 7, 8] # For image data


        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = MessageToDict(hand_landmarks)["landmark"]
                decoded_landmarks = []
                for l in landmarks:
                    x, y, z = int(l['x'] * img_x), int(l['y'] * img_y), l['z']
                    decoded_landmarks.append({'x': x, 'y': y, 'z': z})
                index_finger_landmarks = [decoded_landmarks[i] for i in point_draw]
                landmarks_data.append([classnumber] + [point['x']/img_x for point in index_finger_landmarks] + [point['y']/img_x for point in index_finger_landmarks] + [point['z'] for point in index_finger_landmarks]) # for csv data
                for i in point_draw:
                    cv2.circle(frame, (decoded_landmarks[i]['x'], decoded_landmarks[i]['y']), 5, (255, 255 ,255), 2)
                for c in keypoint_connections:
                    cv2.line(frame, (decoded_landmarks[c[0]]['x'], decoded_landmarks[c[0]]['y']) , (decoded_landmarks[c[1]]['x'], decoded_landmarks[c[1]]['y']), (255, 255, 255), 2)

        cv2.imshow('Hand Landmarks', frame)
        cv2.waitKey(1)

        skeleton = cv2.bitwise_and(frame, cv2.bitwise_not(framewithoutland))
        skeleton = cv2.addWeighted(skeleton, 1, skeleton, 0, 50)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                min_x = min(decoded_landmarks[i]['x'] for i in point_draw)
                min_y = min(decoded_landmarks[i]['y'] for i in point_draw)
                max_x = max(decoded_landmarks[i]['x'] for i in point_draw)
                max_y = max(decoded_landmarks[i]['y'] for i in point_draw)
                real_x_min, real_y_min = int(min_x), int(min_y)
                real_x_max, real_y_max = int(max_x), int(max_y)
                lenght_x = real_x_max - real_x_min
                lenght_y = real_y_max - real_y_min
                if lenght_x > lenght_y:
                    real_y_min -= (lenght_x - lenght_y)/8
                    real_y_max += (lenght_x - lenght_y)/8
                else:
                    real_x_min -= (lenght_y - lenght_x)/8
                    real_x_max += (lenght_y - lenght_x)/8
                #print(real_x_min, real_x_max, real_y_min, real_y_max)
                if real_x_min >= 0 and real_x_max <= img_x and real_y_min >= 0 and real_y_max <= img_y:
                    #hand_cropped = skeleton[int(real_y_min)-10:int(real_y_max)+10, int(real_x_min)-10:int(real_x_max)+10]
                    hand_cropped = framewithoutland[int(real_y_min)-23:int(real_y_max)+23, int(real_x_min)-23:int(real_x_max)+23]
                    if not hand_cropped.size == 0:
                        hand_image = cv2.resize(hand_cropped, image_size)
                        image_path = os.path.join(folder_path_wrong_right, f'finger_{count}.jpg')
                        landmarks_data.append([classnumber] + [point['x']/img_x for point in index_finger_landmarks] + [point['y']/img_y for point in index_finger_landmarks] + [point['z'] for point in index_finger_landmarks])
                        cv2.imwrite(image_path, hand_image) # for image data
                        count += 1


In [12]:
print(chord_shape.value, finger.value, wrong_right.value)

C Middle Right


In [48]:
capture_hand_images_chord(chord_shape.value)

In [95]:
csv_file_path = os.path.join(folder_path, chord_shape.value, f'{chord_shape.value}_landmarks.csv')
with open(csv_file_path, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        header_row = ['Class'] + [f'Landmark_{i+1}_x' for i in range(21)] + [f'Landmark_{i+1}_y' for i in range(21)] + [f'Landmark_{i+1}_z' for i in range(21)] 
        csv_writer.writerow(header_row)
        csv_writer.writerows(landmarks_data)

In [55]:
capture_hand_finger_wrong(finger.value, wrong_right.value)

In [50]:
csv_file_path = 'D:\Chord_Data\Chords\Chords_test_data.csv'
with open(csv_file_path, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        header_row = ['Class'] + [f'Landmark_{i+1}_x' for i in range(21)] + [f'Landmark_{i+1}_y' for i in range(21)] + [f'Landmark_{i+1}_z' for i in range(21)]
        csv_writer.writerow(header_row)
        csv_writer.writerows(landmarks_data)

In [ ]:
csv_file_path = os.path.join(folder_path, finger.value, wrong_right.value, f'{finger.value}_landmarks_finetune.csv')
with open(csv_file_path, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        header_row = ['Class'] + [f'Landmark_{i+1}_x' for i in range(4)] + [f'Landmark_{i+1}_y' for i in range(4)] + [f'Landmark_{i+1}_z' for i in range(4)]
        csv_writer.writerow(header_row)
        csv_writer.writerows(landmarks_data)

In [51]:
cap.release()
cv2.destroyAllWindows()

In [21]:
folder_path_inside = r'D:\Chord_Data\C\Correct'
def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2)
    results = hands.process(rgb_image)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing = mp.solutions.drawing_utils
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    return image